In [1]:
import numpy as np
import pandas as pd
import seaborn as sns

from tennis_model_predict import get_model_win_proba
from tennis_model_predict import general_elo_dict, hard_dict, ts_dict, hand_dict

from trueskillthroughtime import Game, Player

from elo_funcs import elo_predict

from sklearn.metrics import accuracy_score, roc_auc_score, brier_score_loss
from sklearn.calibration import calibration_curve

C:\Users\jyoung\Projects\tennis_project\tennis_model_predict.py:54: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'{data_folder}/processed_apt_with_ts.csv', parse_dates=['tourney_date'])


In [2]:
ts_file = r'C:\Users\jyoung\Projects\tennis_project\tennis_data\tennis_atp-master\processed_apt_with_ts.csv'

ts_df = pd.read_csv(ts_file, parse_dates=['tourney_date'])
# ts_df['winner_last'] = [x.split(' ')[-1] for x in ts_df['winner_name']]
# ts_df['loser_name'] = [x.split(' ')[-1] for x in ts_df['loser_name']]
ts_df['month'] = ts_df['tourney_date'].dt.month
ts_df['year'] = ts_df['tourney_date'].dt.year

C:\Users\jyoung\AppData\Local\Temp\ipykernel_12376\1959857988.py:3: DtypeWarning: Columns (8,16) have mixed types. Specify dtype option on import or set low_memory=False.
  ts_df = pd.read_csv(ts_file, parse_dates=['tourney_date'])


In [3]:
xls_file = r'C:\Users\jyoung\Projects\tennis_project\tennis_data\tennis_atp_betting_results\2022.xlsx'

df = pd.read_excel(xls_file, parse_dates=['Date'])
df.columns = df.columns.str.lower()
df['month'] = df['date'].dt.month
df['year'] = df['date'].dt.year

In [4]:
df['winner_last'] = [x.split(' ')[0] for x in df['winner']]
df['loser_last'] = [x.split(' ')[0] for x in df['loser']]

In [5]:
def true_skill_win_proba(player_1=None, player_2=None):

  player_1 = Player(ts_dict[player_1][-1][1])
  player_2 = Player(ts_dict[player_2][-1][1])

  return Game([[player_1], [player_2]]).evidence

In [6]:
test_df = (
    df
    .merge(ts_df,
           how='left',
           left_on=['year', 'month', 'wrank', 'lrank'],
           right_on=['year', 'month', 'winner_rank', 'loser_rank'])
)

test_df.dropna(subset=['winner_ts_win_prob', 'avgw', 'b365w'], inplace=True)

test_df['avgw_prob'] = 1 / (test_df['avgw'])
test_df['b365w_prob'] = 1 / (test_df['b365w'])

In [7]:
brier_score_loss([1] * len(test_df), test_df['winner_ts_win_prob'])

0.1800769719019201

In [8]:
brier_score_loss([1] * len(test_df), test_df['winner_elo_proba'])

0.2140864277649698

In [12]:
def return_losses(x):
    
    a = accuracy_score([1] * len(x), x['winner_ts_win_prob'].round(0))
    b = accuracy_score([1] * len(x), x['avgw_prob'].round())
    
    x['ts_loss'] = a
    x['market_loss'] = b
    
    return x[['ts_loss', 'market_loss']].reset_index(drop=True).drop_duplicates(subset=['ts_loss'])

In [13]:
test_df.groupby('surface_y').apply(return_losses)

,,ts_loss,market_loss
surface_y,,,
Clay,0,0.720805,0.712752
Grass,0,0.710884,0.687075
Hard,0,0.745605,0.709411


In [39]:
test_df.groupby(['surface_y', 'round_y']).apply(lambda x: brier_score_loss([1] * len(x), x['avgw_prob'])).reset_index()

,surface_y,round_y,0
0,Clay,1.0,0.159116
1,Clay,2.0,0.146755
2,Clay,3.0,0.187112
3,Clay,4.0,0.178616
4,Clay,5.0,0.172318
5,Clay,6.0,0.182899
6,Clay,7.0,0.246965
7,Grass,1.0,0.139426
8,Grass,2.0,0.181314
9,Grass,3.0,0.199222


In [11]:
test_df.groupby(['surface_y', 'round_y']).apply(lambda x: brier_score_loss([1] * len(x), x['winner_ts_win_prob'])).reset_index()

,surface_y,round_y,0
0,Clay,1.0,0.168815
1,Clay,2.0,0.167701
2,Clay,3.0,0.192341
3,Clay,4.0,0.181696
4,Clay,5.0,0.186378
5,Clay,6.0,0.176302
6,Clay,7.0,0.201395
7,Grass,1.0,0.138109
8,Grass,2.0,0.184846
9,Grass,3.0,0.211915


In [25]:
test_df.groupby('pred_bin').apply(lambda x: len(x)/len(test_df))

pred_bin
(0.0, 0.1]    0.001994
(0.1, 0.2]    0.012463
(0.2, 0.3]    0.039382
(0.3, 0.4]    0.090728
(0.4, 0.5]    0.124128
(0.5, 0.6]    0.173480
(0.6, 0.7]    0.186939
(0.7, 0.8]    0.167498
(0.8, 0.9]    0.134596
(0.9, 1.0]    0.068794
dtype: float64